# 네이버 스마트스토어 후기 크롤링 프로그램 (구글 드라이브 저장 버전)

## 프로그램 설명
네이버 스마트스토어 상품 페이지의 리뷰를 자동으로 수집하고, 구글 드라이브에 자동으로 업로드하는 프로그램입니다.

## 주요 기능
- 네이버 스마트스토어 상품 리뷰 자동 수집
- 리뷰 정렬 옵션 지원 (랭킹순, 최신순, 평점 높은순, 평점 낮은순)
- 100개 리뷰마다 중간 저장 기능
- 구글 드라이브 자동 업로드
- 수집 데이터: 평점, 구매자 아이디, 날짜, 선택 옵션명, 리뷰 내용

## 사용 방법
1. 프로그램 실행 후 상품 페이지 URL 입력
2. 저장할 CSV 파일 이름 입력
3. 정렬 방식 선택 (1: 랭킹순, 2: 최신순, 3: 평점 높은순, 4: 평점 낮은순)
4. 크롤링할 최대 리뷰 수 입력
5. 자동으로 크롤링 및 구글 드라이브 업로드 진행

## 필요 라이브러리
- selenium: 웹 크롤링
- pandas: 데이터 처리
- google-api-python-client: 구글 드라이브 API
- webdriver-manager: ChromeDriver 자동 관리

## 주의사항
- Google Drive API 인증이 필요합니다 (client_secret.json 파일 필요)
- 네이버 웹사이트 구조 변경 시 수정이 필요할 수 있습니다
- 과도한 요청은 IP 차단을 유발할 수 있으므로 적절한 딜레이가 설정되어 있습니다

In [3]:
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random
import os
import subprocess
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException, NoSuchWindowException
from datetime import datetime
import sys

# Google Drive API 관련 라이브러리 추가
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 하드 코딩된 구글 드라이브 폴더 ID
GOOGLE_DRIVE_FOLDER_ID = "18SbGtVGs19fYoe-XNajnnl3IlLrP5R2u"


# 사용자 입력 받는 함수
def get_user_input():
    while True:
        url = input("상품 페이지 URL을 입력하세요: ")
        if 'naver.com' in url:
            break
        else:
            logging.warning("URL에 'naver.com'이 포함되어야 합니다. 다시 입력하세요.")
    
    file_name = input("저장할 CSV 파일 이름을 입력하세요: ")

    # 리뷰 정렬 방식 선택
    print("정렬 방식을 선택하세요:")
    print("1: 랭킹순")
    print("2: 최신순")
    print("3: 평점 높은순")
    print("4: 평점 낮은순")
    while True:
        try:
            sort_option = int(input("정렬 옵션 번호를 입력하세요 (1-4): "))
            if 1 <= sort_option <= 4:
                break
            else:
                logging.warning("1에서 4 사이의 번호를 입력해주세요.")
        except ValueError:
            logging.warning("올바른 정수를 입력해주세요.")
    
    while True:
        try:
            max_reviews = int(input("크롤링할 최대 리뷰 수를 입력하세요: "))
            if max_reviews > 0:
                break
            else:
                logging.warning("0보다 큰 정수를 입력해주세요.")
        except ValueError:
            logging.warning("올바른 정수를 입력해주세요.")
    
    return url, file_name, sort_option, max_reviews

# 웹드라이버 초기화 함수
def initialize_webdriver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# 랜덤 대기 시간 함수
def random_wait(min_time=2, max_time=4):
    time.sleep(random.uniform(min_time, max_time))

# 새로고침 버튼 클릭 함수 추가
def check_and_click_refresh_button(driver):
    try:
        # 새로고침 버튼을 찾음
        refresh_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@class, "button highlight") and contains(text(), "새로고침")]'))
        )
        if refresh_button:
            logging.info("새로고침 버튼이 감지되었습니다. 버튼을 클릭합니다.")
            refresh_button.click()
            random_wait()  # 페이지 로드를 기다리는 시간
            logging.info("페이지가 새로고침되었습니다.")
    except TimeoutException:
        logging.info("새로고침 버튼이 감지되지 않았습니다. 계속 진행합니다.")
    except NoSuchElementException:
        logging.info("새로고침 버튼을 찾을 수 없습니다. 계속 진행합니다.")

# 리뷰 정렬 옵션 설정 함수
def set_review_sort(driver, sort_option):
    try:
        if sort_option == 1:
            sort_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "랭킹순")]'))
            )
        elif sort_option == 2:
            sort_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "최신순")]'))
            )
        elif sort_option == 3:
            sort_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "평점 높은순")]'))
            )
        elif sort_option == 4:
            sort_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "평점 낮은순")]'))
            )
        sort_button.click()
        logging.info(f"선택한 정렬 옵션: {['랭킹순', '최신순', '평점 높은순', '평점 낮은순'][sort_option - 1]}")
        random_wait()
    except (NoSuchElementException, TimeoutException) as e:
        logging.error(f"정렬 버튼을 클릭할 수 없습니다. 오류: {e}")

# 리뷰 데이터를 중간 저장하는 함수
def save_to_csv_partial(reviews_data, file_name, sort_option, part_number):
    sort_options = {1: "랭킹순", 2: "최신순", 3: "평점높은순", 4: "평점낮은순"}
    sort_option_name = sort_options[sort_option]
    
    current_date = datetime.now().strftime("%Y-%m-%d")
    final_file_name = f"{file_name}_스마트스토어후기_{sort_option_name}_{current_date}_part{part_number}"
    
    df = pd.DataFrame(reviews_data)
    df.index += 1
    full_path = os.path.abspath(f"{final_file_name}.csv")
    df.to_csv(full_path, index_label="순번", encoding="utf-8-sig")
    logging.info(f"{full_path} 파일에 중간 저장되었습니다.")
    return full_path

# 리뷰 데이터를 크롤링하는 함수 (중간 저장 추가)
def extract_review_info(driver, reviews_data, max_reviews, file_name, sort_option):
    page_num = 1
    part_number = 1  # 저장할 파트 번호
    while len(reviews_data) < max_reviews:
        try:
            logging.info(f"현재 페이지: {page_num}")
            
            reviews = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.BnwL_cs1av'))
            )
            
            logging.info(f"현재 페이지에서 발견된 리뷰 수: {len(reviews)}")
            
            for review in reviews:
                if len(reviews_data) >= max_reviews:
                    break
                try:
                    rating = review.find_element(By.CSS_SELECTOR, 'em._15NU42F3kT').text
                    user_id = review.find_element(By.CSS_SELECTOR, 'strong._2L3vDiadT9').text
                    raw_date = review.find_element(By.CSS_SELECTOR, 'div.iWGqB6S4Lq span._2L3vDiadT9').text

                    # 날짜 변환 (yy.mm.dd -> yyyy-mm-dd)
                    raw_date = raw_date.split('.')[0:3]
                    date_str = '.'.join(raw_date)
                    date = datetime.strptime(date_str, "%y.%m.%d").strftime("%Y-%m-%d")
                    
                    option = review.find_element(By.CSS_SELECTOR, 'div._2FXNMst_ak').text
                    review_text = review.find_element(By.CSS_SELECTOR, 'div._1kMfD5ErZ6 span._2L3vDiadT9').text
                    
                    reviews_data.append({
                        "평점": rating,
                        "구매자아이디": user_id,
                        "날짜": date,
                        "선택옵션명": option,
                        "리뷰내용": review_text
                    })
                    logging.info(f"리뷰 {len(reviews_data)}: {user_id} - {rating}")
                except NoSuchElementException as e:
                    logging.warning(f"리뷰 요소를 찾을 수 없습니다. 오류: {e}")
                    continue

            # 100개의 리뷰마다 중간 저장
            if len(reviews_data) % 100 == 0:
                save_to_csv_partial(reviews_data, file_name, sort_option, part_number)
                part_number += 1  # 다음 저장 파트를 위해 번호 증가

            # 다음 페이지로 이동
            try:
                next_page_buttons = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.UWN4IvaQza._nlog_click[aria-current="false"]'))
                )
                
                next_page_button = None
                for button in next_page_buttons:
                    button_number = int(button.text)
                    if button_number > page_num:
                        next_page_button = button
                        break
                
                if next_page_button:
                    driver.execute_script("arguments[0].click();", next_page_button)
                    page_num = button_number
                    random_wait()
                else:
                    logging.info("다음 페이지 버튼을 찾을 수 없습니다. 마지막 페이지일 수 있습니다.")
                    break
            except (TimeoutException, NoSuchElementException) as e:
                logging.error(f"다음 페이지 버튼을 찾을 수 없습니다. 오류: {e}")
                break

        except TimeoutException:
            logging.error("페이지 로드 시간이 초과되었습니다. 다음 페이지로 넘어갑니다.")
            page_num += 1
            continue
        except NoSuchWindowException:
            logging.error("브라우저 창이 닫혔습니다. 프로그램을 종료합니다.")
            break
        except Exception as e:
            logging.error(f"예상치 못한 오류가 발생했습니다: {e}")
            break
    
    logging.info(f"총 {len(reviews_data)}개의 리뷰를 수집했습니다.")

# 파일을 시스템에서 자동으로 여는 함수
def open_file(file_path):
    if os.name == 'nt':  # Windows
        os.startfile(file_path)
    elif os.name == 'posix':  # macOS and Linux
        if sys.platform == 'darwin':  # macOS
            subprocess.call(('open', file_path))
        else:  # Linux
            subprocess.call(('xdg-open', file_path))

# Google Drive 업로드 함수
def upload_to_google_drive(file_path, folder_id):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': os.path.basename(file_path), 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    logging.info(f"파일이 구글 드라이브에 업로드되었습니다. File ID: {file.get('id')}")

# 메인 함수 수정
def main():
    url, file_name, sort_option, max_reviews = get_user_input()
    
    driver = initialize_webdriver()
    driver.get(url)

    # 페이지 로드 후 새로고침 버튼 체크 및 클릭
    check_and_click_refresh_button(driver)

    try:
        # 리뷰 탭 클릭
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-name="REVIEW"]'))
        )
        review_tab.click()
        random_wait()

        # 정렬 방식 설정
        set_review_sort(driver, sort_option)

    except (NoSuchElementException, TimeoutException):
        logging.error("리뷰 탭을 찾을 수 없거나 클릭할 수 없습니다.")
        driver.quit()
        return

    reviews_data = []
    file_path = None  # 파일 경로 변수를 미리 선언
    try:
        extract_review_info(driver, reviews_data, max_reviews, file_name, sort_option)
        if reviews_data:
            file_path = save_to_csv_partial(reviews_data, file_name, sort_option, 1)  # 마지막 저장
            logging.info(f"프로그램 종료. 수집된 {len(reviews_data)}개의 리뷰를 저장했습니다.")
            
            # 구글 드라이브에 업로드 (하드 코딩된 폴더 ID 사용)
            upload_to_google_drive(file_path, GOOGLE_DRIVE_FOLDER_ID)
    finally:
        driver.quit()

    if file_path:
        try:
            open_file(file_path)
            logging.info(f"{file_path} 파일이 자동으로 열렸습니다.")
        except Exception as e:
            logging.error(f"파일을 자동으로 열 수 없습니다. 오류: {e}")
    else:
        logging.info("수집된 리뷰가 없으므로 파일을 생성하지 않았습니다.")

if __name__ == "__main__":
    main()

상품 페이지 URL을 입력하세요: https://smartstore.naver.com/ancalls-yarn/products/282535301
저장할 CSV 파일 이름을 입력하세요: 앵콜스_나무 줄바늘
정렬 방식을 선택하세요:
1: 랭킹순
2: 최신순
3: 평점 높은순
4: 평점 낮은순
정렬 옵션 번호를 입력하세요 (1-4): 2
크롤링할 최대 리뷰 수를 입력하세요: 5795


2024-09-19 21:07:25,184 - INFO - ====== WebDriver manager ======
2024-09-19 21:07:25,841 - INFO - Get LATEST chromedriver version for google-chrome
2024-09-19 21:07:26,241 - INFO - Get LATEST chromedriver version for google-chrome
2024-09-19 21:07:26,736 - INFO - Driver [C:\Users\owner\.wdm\drivers\chromedriver\win64\128.0.6613.137\chromedriver-win32/chromedriver.exe] found in cache
2024-09-19 21:07:27,978 - INFO - 새로고침 버튼이 감지되었습니다. 버튼을 클릭합니다.
2024-09-19 21:07:30,890 - INFO - 페이지가 새로고침되었습니다.
2024-09-19 21:07:33,826 - INFO - 선택한 정렬 옵션: 최신순
2024-09-19 21:07:37,010 - INFO - 현재 페이지: 1
2024-09-19 21:07:37,020 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:07:37,077 - INFO - 리뷰 1: puty**** - 5
2024-09-19 21:07:37,117 - INFO - 리뷰 2: puty**** - 5
2024-09-19 21:07:37,156 - INFO - 리뷰 3: hes0**** - 5
2024-09-19 21:07:37,197 - INFO - 리뷰 4: elis***** - 5
2024-09-19 21:07:37,242 - INFO - 리뷰 5: dnjs******* - 5
2024-09-19 21:07:37,287 - INFO - 리뷰 6: hanl******* - 3
2024-09-19 21:07:37,328 - INFO - 리뷰 7:

2024-09-19 21:07:59,213 - INFO - 리뷰 129: wee_**** - 5
2024-09-19 21:07:59,253 - INFO - 리뷰 130: skdu***** - 5
2024-09-19 21:07:59,295 - INFO - 리뷰 131: vv**** - 5
2024-09-19 21:07:59,334 - INFO - 리뷰 132: sooy****** - 5
2024-09-19 21:07:59,374 - INFO - 리뷰 133: sooy****** - 5
2024-09-19 21:07:59,413 - INFO - 리뷰 134: yeow******* - 5
2024-09-19 21:07:59,473 - INFO - 리뷰 135: harm******* - 5
2024-09-19 21:07:59,515 - INFO - 리뷰 136: harm******* - 5
2024-09-19 21:07:59,554 - INFO - 리뷰 137: harm******* - 5
2024-09-19 21:07:59,593 - INFO - 리뷰 138: jess******* - 5
2024-09-19 21:07:59,633 - INFO - 리뷰 139: snow********** - 5
2024-09-19 21:07:59,675 - INFO - 리뷰 140: seoy******** - 5
2024-09-19 21:08:02,532 - INFO - 현재 페이지: 8
2024-09-19 21:08:02,553 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:08:02,594 - INFO - 리뷰 141: jiwo***** - 5
2024-09-19 21:08:02,633 - INFO - 리뷰 142: soun****** - 3
2024-09-19 21:08:02,672 - INFO - 리뷰 143: soun****** - 4
2024-09-19 21:08:02,712 - INFO - 리뷰 144: suga******* - 5
20

2024-09-19 21:08:29,363 - INFO - 리뷰 265: ma**** - 5
2024-09-19 21:08:29,402 - INFO - 리뷰 266: jis0**** - 5
2024-09-19 21:08:29,455 - INFO - 리뷰 267: kwoa****** - 5
2024-09-19 21:08:29,495 - INFO - 리뷰 268: jyy9*** - 5
2024-09-19 21:08:29,536 - INFO - 리뷰 269: jjan***** - 5
2024-09-19 21:08:29,576 - INFO - 리뷰 270: jjan***** - 5
2024-09-19 21:08:29,616 - INFO - 리뷰 271: ar**** - 5
2024-09-19 21:08:29,655 - INFO - 리뷰 272: wwjd**** - 5
2024-09-19 21:08:29,698 - INFO - 리뷰 273: anne**** - 5
2024-09-19 21:08:29,741 - INFO - 리뷰 274: jisy*** - 4
2024-09-19 21:08:29,794 - INFO - 리뷰 275: ydjx*** - 5
2024-09-19 21:08:29,837 - INFO - 리뷰 276: ydjx*** - 5
2024-09-19 21:08:29,878 - INFO - 리뷰 277: ydjx*** - 5
2024-09-19 21:08:29,919 - INFO - 리뷰 278: hs**** - 5
2024-09-19 21:08:29,960 - INFO - 리뷰 279: ch71*** - 5
2024-09-19 21:08:30,002 - INFO - 리뷰 280: lsj9**** - 5
2024-09-19 21:08:32,053 - INFO - 현재 페이지: 15
2024-09-19 21:08:32,081 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:08:32,121 - INFO - 리뷰 281: lsj9

2024-09-19 21:08:56,221 - INFO - 현재 페이지: 21
2024-09-19 21:08:56,232 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:08:56,272 - INFO - 리뷰 401: 48**** - 5
2024-09-19 21:08:56,316 - INFO - 리뷰 402: 48**** - 5
2024-09-19 21:08:56,357 - INFO - 리뷰 403: yuri****** - 5
2024-09-19 21:08:56,396 - INFO - 리뷰 404: snow******** - 5
2024-09-19 21:08:56,436 - INFO - 리뷰 405: ehdg******* - 5
2024-09-19 21:08:56,475 - INFO - 리뷰 406: gksd******* - 5
2024-09-19 21:08:56,515 - INFO - 리뷰 407: 2s**** - 5
2024-09-19 21:08:56,556 - INFO - 리뷰 408: nilb*** - 4
2024-09-19 21:08:56,596 - INFO - 리뷰 409: kkak****** - 5
2024-09-19 21:08:56,635 - INFO - 리뷰 410: kkak****** - 5
2024-09-19 21:08:56,691 - INFO - 리뷰 411: kkak****** - 5
2024-09-19 21:08:56,732 - INFO - 리뷰 412: wets**** - 5
2024-09-19 21:08:56,771 - INFO - 리뷰 413: simo***** - 5
2024-09-19 21:08:56,811 - INFO - 리뷰 414: simo***** - 5
2024-09-19 21:08:56,851 - INFO - 리뷰 415: re**** - 5
2024-09-19 21:08:56,890 - INFO - 리뷰 416: kbj4*** - 5
2024-09-19 21:08:56,930 - I

...

2024-09-19 21:24:40,170 - INFO - 리뷰 5204: bbio*** - 5
2024-09-19 21:24:40,210 - INFO - 리뷰 5205: bbio*** - 5
2024-09-19 21:24:40,249 - INFO - 리뷰 5206: bbio*** - 5
2024-09-19 21:24:40,289 - INFO - 리뷰 5207: bbio*** - 5
2024-09-19 21:24:40,327 - INFO - 리뷰 5208: bbio*** - 5
2024-09-19 21:24:40,367 - INFO - 리뷰 5209: bbio*** - 5
2024-09-19 21:24:40,409 - INFO - 리뷰 5210: yoon**** - 5
2024-09-19 21:24:40,451 - INFO - 리뷰 5211: goql*** - 4
2024-09-19 21:24:40,492 - INFO - 리뷰 5212: goql*** - 5
2024-09-19 21:24:40,532 - INFO - 리뷰 5213: en**** - 5
2024-09-19 21:24:40,570 - INFO - 리뷰 5214: jiyo******* - 5
2024-09-19 21:24:40,610 - INFO - 리뷰 5215: best******* - 5
2024-09-19 21:24:40,649 - INFO - 리뷰 5216: 33**** - 5
2024-09-19 21:24:40,688 - INFO - 리뷰 5217: zlzl**** - 5
2024-09-19 21:24:40,728 - INFO - 리뷰 5218: zlzl**** - 5
2024-09-19 21:24:40,765 - INFO - 리뷰 5219: suju****** - 3
2024-09-19 21:24:40,806 - INFO - 리뷰 5220: sa18**** - 5
2024-09-19 21:24:44,803 - INFO - 현재 페이지: 262
2024-09-19 21:24:44,810 

2024-09-19 21:25:04,257 - INFO - 리뷰 5339: jya7*** - 5
2024-09-19 21:25:04,297 - INFO - 리뷰 5340: 6644*** - 5
2024-09-19 21:25:07,608 - INFO - 현재 페이지: 268
2024-09-19 21:25:07,616 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:25:07,658 - INFO - 리뷰 5341: 6644*** - 5
2024-09-19 21:25:07,696 - INFO - 리뷰 5342: rose****** - 3
2024-09-19 21:25:07,734 - INFO - 리뷰 5343: rose****** - 4
2024-09-19 21:25:07,774 - INFO - 리뷰 5344: rose****** - 4
2024-09-19 21:25:07,813 - INFO - 리뷰 5345: rose****** - 2
2024-09-19 21:25:07,853 - INFO - 리뷰 5346: rose****** - 5
2024-09-19 21:25:07,895 - INFO - 리뷰 5347: rose****** - 5
2024-09-19 21:25:07,936 - INFO - 리뷰 5348: rose****** - 5
2024-09-19 21:25:07,974 - INFO - 리뷰 5349: rose****** - 5
2024-09-19 21:25:08,015 - INFO - 리뷰 5350: co**** - 3
2024-09-19 21:25:08,056 - INFO - 리뷰 5351: co**** - 3
2024-09-19 21:25:08,094 - INFO - 리뷰 5352: rs**** - 4
2024-09-19 21:25:08,133 - INFO - 리뷰 5353: rs**** - 3
2024-09-19 21:25:08,172 - INFO - 리뷰 5354: luck***** - 3
2024-09-19 21:

2024-09-19 21:25:29,783 - INFO - 리뷰 5473: lamc**** - 5
2024-09-19 21:25:29,822 - INFO - 리뷰 5474: lamc**** - 5
2024-09-19 21:25:29,863 - INFO - 리뷰 5475: mkki**** - 5
2024-09-19 21:25:29,902 - INFO - 리뷰 5476: nesa******* - 5
2024-09-19 21:25:29,942 - INFO - 리뷰 5477: nesa******* - 5
2024-09-19 21:25:29,981 - INFO - 리뷰 5478: choy**** - 5
2024-09-19 21:25:30,021 - INFO - 리뷰 5479: tmzl***** - 5
2024-09-19 21:25:30,059 - INFO - 리뷰 5480: yu**** - 5
2024-09-19 21:25:33,457 - INFO - 현재 페이지: 275
2024-09-19 21:25:33,473 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:25:33,513 - INFO - 리뷰 5481: yu**** - 5
2024-09-19 21:25:33,553 - INFO - 리뷰 5482: 75**** - 5
2024-09-19 21:25:33,593 - INFO - 리뷰 5483: 75**** - 5
2024-09-19 21:25:33,632 - INFO - 리뷰 5484: tjdg****** - 5
2024-09-19 21:25:33,670 - INFO - 리뷰 5485: smh9*** - 5
2024-09-19 21:25:33,709 - INFO - 리뷰 5486: smar***** - 5
2024-09-19 21:25:33,748 - INFO - 리뷰 5487: qngh******* - 3
2024-09-19 21:25:33,788 - INFO - 리뷰 5488: qngh******* - 3
2024-09-19 21

2024-09-19 21:25:55,826 - INFO - 리뷰 5604: go**** - 5
2024-09-19 21:25:55,870 - INFO - 리뷰 5605: go**** - 5
2024-09-19 21:25:55,913 - INFO - 리뷰 5606: go**** - 5
2024-09-19 21:25:55,956 - INFO - 리뷰 5607: eunj******** - 5
2024-09-19 21:25:56,000 - INFO - 리뷰 5608: eunj******** - 5
2024-09-19 21:25:56,042 - INFO - 리뷰 5609: 9924*** - 5
2024-09-19 21:25:56,084 - INFO - 리뷰 5610: 9924*** - 5
2024-09-19 21:25:56,125 - INFO - 리뷰 5611: 9924*** - 5
2024-09-19 21:25:56,167 - INFO - 리뷰 5612: wkwn**** - 5
2024-09-19 21:25:56,208 - INFO - 리뷰 5613: og**** - 5
2024-09-19 21:25:56,249 - INFO - 리뷰 5614: tv**** - 5
2024-09-19 21:25:56,288 - INFO - 리뷰 5615: ma**** - 5
2024-09-19 21:25:56,326 - INFO - 리뷰 5616: hych**** - 5
2024-09-19 21:25:56,365 - INFO - 리뷰 5617: 0169******* - 5
2024-09-19 21:25:56,405 - INFO - 리뷰 5618: 0169******* - 5
2024-09-19 21:25:56,444 - INFO - 리뷰 5619: 0169******* - 5
2024-09-19 21:25:56,483 - INFO - 리뷰 5620: 0169******* - 5
2024-09-19 21:25:58,941 - INFO - 현재 페이지: 282
2024-09-19 21:2

2024-09-19 21:26:19,833 - INFO - 리뷰 5739: arie***** - 5
2024-09-19 21:26:19,874 - INFO - 리뷰 5740: wkdi***** - 5
2024-09-19 21:26:23,185 - INFO - 현재 페이지: 288
2024-09-19 21:26:23,191 - INFO - 현재 페이지에서 발견된 리뷰 수: 20
2024-09-19 21:26:23,232 - INFO - 리뷰 5741: pmh5*** - 5
2024-09-19 21:26:23,271 - INFO - 리뷰 5742: pmh5*** - 5
2024-09-19 21:26:23,310 - INFO - 리뷰 5743: pmh5*** - 5
2024-09-19 21:26:23,350 - INFO - 리뷰 5744: pmh5*** - 5
2024-09-19 21:26:23,388 - INFO - 리뷰 5745: fofo*** - 5
2024-09-19 21:26:23,427 - INFO - 리뷰 5746: fofo*** - 5
2024-09-19 21:26:23,465 - INFO - 리뷰 5747: chri*** - 5
2024-09-19 21:26:23,506 - INFO - 리뷰 5748: chri*** - 5
2024-09-19 21:26:23,546 - INFO - 리뷰 5749: ki**** - 5
2024-09-19 21:26:23,584 - INFO - 리뷰 5750: ki**** - 5
2024-09-19 21:26:23,623 - INFO - 리뷰 5751: srpa****** - 5
2024-09-19 21:26:23,661 - INFO - 리뷰 5752: srpa****** - 5
2024-09-19 21:26:23,729 - INFO - 리뷰 5753: ting******** - 5
2024-09-19 21:26:23,916 - INFO - 리뷰 5754: year*** - 5
2024-09-19 21:26:23,957